# CS448 - Lab 8: Audio Classification 

## Part 1: Making a speech detector

In this section we will design a simple classifier that will let us know if its input is speech or non-speech. Download the data archive from: [ https://drive.google.com/file/d/1Z8tj-HHQCvT54Mr20Dc1UfV48SPY449r/view ] In this part we will use the dataset in data/SpeechMusic. In it you will find two directories, speech/ and music/ containing data from each class.

Randomly select 50 soundfiles from each directory to use as training data, and use the remaining sounds as testing data. For all of the sounds we will compute a representation that makes the classification easier and we will use a simple Gaussian model to classify them. Do the following:

- Perform an STFT for each sound, take it’s magnitude and raise it to 0.3 to improve contrast
    - We will consider each spectral slice of that to be a data point
- Using the training data of each sound:
    - Calculate the mean column and the diagonal covariance of the columns
    - You will thus get two sets of Gaussian parameters that model each sound class
- For each testing data point:
    - Calculate the likelihood of each column based on the above models
	- To calculate the entire file likelihood add all the frame likelihoods
	- Assign each soundfile to the class that gets the highest likelihood

For extra credit implement the parameter estimation and model likelihood yourself. If you are too lazy for that you can instead use ```sklearn.mixture.GaussianMixture``` to learn a diagonal single-Gaussian model per class.

How do the results look like? If you rerun this with a different training/testing set, is there an appreciable difference? On average over multiple training/testing sets what accuracy do you get?

In [39]:
import scipy
from scipy import signal
import numpy as np
from numpy import min, max, remainder, fromfile, abs
import matplotlib.pyplot as plt
import pdb
import math
import cmath
from sklearn.mixture import GaussianMixture
from os import listdir
import os
 
def wavreadurl( url):
    import urllib.request, io, scipy.io.wavfile
    f = urllib.request.urlopen( urllib.request.Request( url))
    sr,s = scipy.io.wavfile.read( io.BytesIO( f.read()))
    return sr, s.astype( 'float32')/32768

def wavreadlocal(path):
    import urllib.request, io, scipy.io.wavfile
    sr,s = scipy.io.wavfile.read(path)
    return sr, s.astype( 'float32')/32768

def sound( x, rate=8000, label=''):
    from IPython.display import display, Audio, HTML
    display( HTML( 
    '<style> table, th, td {border: 0px; }</style> <table><tr><td>' + label + 
    '</td><td>' + Audio( x, rate=rate)._repr_html_()[3:] + '</td></tr></table>'
    ))

def stft( input_sound, dft_size, hop_size, zero_pad, window):
    input_filled = input_sound
    
    # Fill frames list with windowed vectors
    frames_list = []
    for i in range(int(len(input_filled) / hop_size) - 2):
        if (i*hop_size+dft_size < len(input_filled)):
            frames_list.append(input_filled[(i*hop_size):(i*hop_size)+dft_size])
            
    windowed_list = []
    for frame in frames_list:
        windowed_list.append(np.asarray([frame[i] * window[i] for i in range(dft_size)]))
    
    # Take all of the DFT vectors and concatenate them as columns of a matrix.
    spectro = []
    for x in windowed_list:
        spectro.append(np.fft.rfft(x, dft_size + zero_pad))
    
    # Return a complex-valued spectrogram (frequencies x time)
    return np.array(spectro)

def istft( stft_output, dft_size, hop_size, zero_pad):
    windowed_list = []
    for x in stft_output.T:
        windowed_list.append(np.fft.irfft(x, dft_size + zero_pad))
    windowed_list = np.array(windowed_list)
    
    windowed_list = windowed_list.T[:dft_size].T

    # Length of Array: hop_size * (len(windowed_list) - 1) + dft_size
    fill = np.zeros(hop_size * (len(windowed_list) - 1) + dft_size)
    for i in range(len(windowed_list)): # Iterate Frames
            fill[(i*hop_size):(i*hop_size)+dft_size] += windowed_list[i]

    # Return reconstructed waveform
    return fill

def createXTicks(stft_output, sr):
    x_ind = []
    for i in range(len(stft_output.T)):
        x_ind.append('%.2f'%(128 * i / sr))
    return x_ind

def createYTicks(stft_output, sr):
    y_ind = []
    for i in range(len(stft_output)):
        y_ind.append(int(i * sr / 512))
    return y_ind

# Plot all the spectrograms
def plotSpectrogram(stft, sr, ntitle=""):
    plt.figure()
    plt.pcolormesh(np.abs(stft)**0.2)
    plt.title(ntitle)
    plt.xlabel("Seconds")
    plt.ylabel("Frequency")
    plt.xticks(range(len(stft.T)), createXTicks(stft, sr))
    plt.yticks(range(len(stft)), createYTicks(stft,sr))
    plt.locator_params(axis='x', nbins=10)
    plt.locator_params(axis='y', nbins=12)

In [2]:
# YOUR CODE HERE
# Perform an STFT for each sound, take it’s magnitude and raise it to 0.3 to improve contrast
# We will consider each spectral slice of that to be a data point
speech_data = []
for file in listdir("data/data/SpeechMusic/speech"):
    sr, wav = wavreadlocal("data/data/SpeechMusic/speech/"+file)
    stft_result = np.absolute(stft(wav, 2048, 256, 0, np.hanning(2048)))**0.3
    speech_data.append(stft_result)
    
music_data = []
for file in listdir("data/data/SpeechMusic/music"):
    sr, wav = wavreadlocal("data/data/SpeechMusic/music/"+file)
    stft_result = np.absolute(stft(wav, 2048, 256, 0, np.hanning(2048)))**0.3
    music_data.append(stft_result)

print("done")

done


In [16]:
import random
# Using the training data of each sound:
# Calculate the mean column and the diagonal covariance of the columns
# You will thus get two sets of Gaussian parameters that model each sound class

all_speeches = speech_data.copy()
random.shuffle(all_speeches)
speech_training_set = all_speeches[:50]
speech_testing_set = all_speeches[50:]

all_music = music_data.copy()
random.shuffle(all_music)
music_training_set = all_music[:50]
music_testing_set = all_music[50:]

gauss_speech = GaussianMixture(1, covariance_type="diag").fit(np.concatenate(speech_training_set))
gauss_music = GaussianMixture(1, covariance_type="diag").fit(np.concatenate(music_training_set))

# For each testing data point:
# Calculate the likelihood of each column based on the above models
# To calculate the entire file likelihood add all the frame likelihoods
# Assign each soundfile to the class that gets the highest likelihood

speech_correct = 0
for test in speech_testing_set:
    if (gauss_speech.score(test) > gauss_music.score(test)):
        speech_correct += 1
        
music_correct = 0
for test in music_testing_set:
    if (gauss_music.score(test) > gauss_speech.score(test)):
        music_correct += 1

print("Accuracy", (speech_correct + music_correct) / 20)
print("The accuracy varies from 0.7 to 0.9 depending on the training/testing sets.")

Accuracy 0.85
The accuracy varies from 0.7 to 0.9 depending on the training/testing sets.


## Part 2: Making a music genre classifier

We will repeat the above, but this time we will perform music genre classification. To do so we will use a slightly more elaborate feature representation, and a stronger classification model. If you downloaded the data archive pointed to above, you will find a subset of the CTZAN dataset in the data/genre folder, this is a benchmark data set for music genre classification.

Just as before, you will find a set of directories with examples of each sound class that we want to recognize. For each class, split the soundfiles into a training set (50% of data) and testing set (remaining 50% of data).

For a representation we will use MFCC features. For extra credit, code these yourself otherwise you can use the implementation from the ```librosa``` library. Once all the files are transformed we will have a series of MFCC frames for each recording (as opposed to spectral frames as is in the case of the STFT). We will use these as the data to classify.

For each class learn a Gaussian model (with a diagonal covariance again). This will be the same process as above.
In order to evaluate how good this works we will use the following procedure. For each sound in the training data, get the likelihood of each MFCC frame based on the learned Gaussian models and sum these over the entire file just as we did before. Use the resulting values to get a classification result for each . Report how accurate your results are. Now report the accuracy using your testing data instead.

Now will use a better classifier to hopefully get better accuracy. We will use a Gaussian Mixture Model (```sklearn.mixture.GaussianMixture```). Just as before you should learn one such model for each class using the corresponding training data.

How many Gaussians do you need in your GMM to get the best results? Do the MFCC parameters make a difference? Play around with the numbers to get the best possible results.  You should be able to get at least 70% accuracy on average.

In [34]:
import librosa
def get_genre_data(genre):
    data = []
    for file in listdir("data/data/genres/"+genre):
        file, sr = librosa.core.load("data/data/genres/"+genre+"/"+file)
        file = np.array(file)
        feature = librosa.feature.mfcc(y=file, sr=sr, n_mfcc=60)
        data.append(feature)
    return data
    
classical_data = get_genre_data('classical')
disco_data = get_genre_data('disco')
metal_data = get_genre_data('metal')
pop_data = get_genre_data('pop')
reggae_data = get_genre_data('reggae')

In [69]:
import random
def splitter(all_samples):
    all_samples = all_samples.copy()
    random.shuffle(all_samples)
    index = int(len(all_samples)/2)
    training_set = all_samples[:index]
    testing_set = all_samples[index:]
    return training_set, testing_set

In [72]:
classical_training, classical_testing = splitter(classical_data)
disco_training, disco_testing = splitter(disco_data)
metal_training, metal_testing = splitter(metal_data)
pop_training, pop_testing = splitter(pop_data)
reggae_training, reggae_testing = splitter(reggae_data)

gauss_classical = GaussianMixture(5, covariance_type="diag").fit(np.concatenate(classical_training, axis=1).T)
gauss_disco = GaussianMixture(5, covariance_type="diag").fit(np.concatenate(disco_training, axis=1).T)
gauss_metal = GaussianMixture(5, covariance_type="diag").fit(np.concatenate(metal_training, axis=1).T)
gauss_pop = GaussianMixture(5, covariance_type="diag").fit(np.concatenate(pop_training, axis=1).T)
gauss_reggae = GaussianMixture(5, covariance_type="diag").fit(np.concatenate(reggae_training, axis=1).T)

In [75]:
num_correct = 0
genres_testing = [classical_testing, disco_testing, metal_testing, pop_testing, reggae_testing]
genres_training = [gauss_classical, gauss_disco, gauss_metal, gauss_pop, gauss_reggae]

for curr, genre in enumerate(genres_testing):
    for sample in genre:
        llhs = []
        for train in genres_training:
            llhs.append(train.score(sample.T))
        if (np.argmax(llhs) == curr):
            num_correct += 1
    
total_tests = 0
for tests in genres_testing:
    total_tests += len(tests)
print("Accuracy: ", num_correct / total_tests)
print("After several rounds of testing, the overall accuracy varies from around 0.82 to 0.87.")

Accuracy:  0.848
After several rounds of testing, the overall accuracy varies from around 0.82 to 0.87.


## Part 3: Make it better (extra credit, required for 4-hour registrants)

There is no shortage of techniques (and free code) to use for classification. Revisit the two problems above and use any other type of classifier you want (Neural Nets, Boosting, Decision Trees, whatever). Also feel free to use any feature you want. Can you improve on the results you got before? How much higher can you get your accuracy for either case?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()